In [ ]:
!pip install -r requirements.txt

In [ ]:
import os
from bingX import BingX
import mplfinance as mpl
import numpy as np
import pandas as pd
from datetime import datetime
from scipy.signal import argrelextrema
import matplotlib.pyplot as plt
import pandas_ta as ta

TRADE_INTERVALS = ["1M","1d","4h","1h","30m","15m","5m","3m","1m"]

# Exchange
API_KEY = os.environ.get("BINGX_API_KEY","")
SECRET_KEY = os.environ.get("BINGX_SECRET_KEY","")
bingx_client = BingX(API_KEY, SECRET_KEY)

def get_data(symbol, interval, start, end):
    start_time_ms = int(start.timestamp() * 1000)
    end_time_ms = int(end.timestamp() * 1000)
    data = bingx_client.perpetual_v2.market.get_k_line_data(symbol=symbol,interval=interval,start_time=start_time_ms,end_time=end_time_ms)
    df = pd.DataFrame(data)
    df.set_index('time', inplace=True)
    df.index = pd.to_datetime(df.index, unit='ms')
    df = df.astype(float)
    return df


def get_levels(date):
    ratios = [-0.618, 0.618, 1.618] # SL - Entry - TP
    series_ = df.loc[date:][1:2].squeeze()
    diff = series_.high - series_.low
    levels = [series_.close + i * diff for i in ratios]
    return levels

def get_median_values(lf):
    n = 5
    lf['median'] = np.abs(lf['high'] + lf['low']) / 2
    lf['min'] = lf.iloc[argrelextrema(lf['median'].values, np.less_equal,
                                      order=n)[0]]['median']
    lf['max'] = lf.iloc[argrelextrema(lf['median'].values, np.greater_equal,
                                      order=n)[0]]['median']
    return lf

# Date intervals to fetch data
date = datetime(2022,1,1,0,0,0)
date_end = datetime(2022,1,5,0,0,0)
df = get_data("BTC-USDT", "1h", date, date_end)
lf = get_median_values(df)

#lf = df.drop(columns=['volume','open','close'])
#pdf = lf.drop(columns=['high','low'])
plt.scatter(lf.index, lf['min'], c='r')
for index, min_price in lf['min'].items():
    plt.text(index, min_price, f"{min_price:.2f}", fontsize=8, ha='right', va='bottom', color='r')
plt.scatter(lf.index, lf['max'], c='g')
for index, max_price in lf['max'].items():
    plt.text(index, max_price, f"{max_price:.2f}", fontsize=8, ha='right', va='bottom', color='g')
plt.plot(lf.index, lf['median'])
plt.show()

In [ ]:
# Fibo back testing (WIP) - Still checking if this make sense.
buys,sells = [],[]
trade_dates = []
in_position = False
df['price'] = df.open.shift(-1)
days_range = np.unique(df.index.date)
for date in days_range:
    for index, row in df[date:][2:].iterrows():
        if not in_position:
            sl,entry,tp = get_levels(date)
            if row.close >= entry:
                df.at[index, 'buy'] = row.price
                buys.append(row.price)
                trade_dates.append(date)
                in_position = True
        if in_position:
            if row.close >= tp or row.close <= sl:
                df.at[index, 'sell'] = row.price
                sells.append(row.price)
                in_position = False

trades = pd.DataFrame([buys,sells])
trades.columns = trade_dates
trades.index = ['Buy','Sell']
trades = trades.T


In [ ]:
def fibonacci_levels(max_val, min_val):
    ratios = [-0.618, 0.618, 1.618] # SL - Entry - TP
    diff = max_val - min_val
    levels = [max_val - i * diff for i in ratios]
    return levels

In [ ]:
filtered_df = lf[lf['min'].notnull() | lf['max'].notnull()]
filtered_df

In [ ]:
min_max_vals = []
min_max_type = {0:[],1:[]}
def get_min_max_data():
    idx = 0
    while(idx < len(filtered_df)):
        row = filtered_df.iloc[idx]
        if not pd.isna(row['min']):
            min_max_vals.append(row['min'])
            min_max_type[0].append(row['min'])
        if not pd.isna(row['max']):
            min_max_vals.append(row['max'])
            min_max_type[1].append(row['max'])
        idx = idx + 1

def fibonacci_levels_v2(min_val, max_val):
    diff = max_val - min_val
    fib_5 = min_val + 0.5 * (diff)
    fib_618 = min_val + 0.618 * (diff)
    fib_786 = min_val + 0.786 * (diff)
    return fib_5, fib_618, fib_786

def possible_retracement(price, min_val, max_val):
    fib_5, fib_618, fib_786 = fibonacci_levels_v2(min_val, max_val)
    if fib_618 < price <= fib_786:
        print(f"The price {price} is within the range of 0.618 - 0.786 ({fib_618}, {fib_786})")
    elif fib_5 < price <= fib_786:  # It seems like 'fib_5' is undefined here. Perhaps it should be 'fib_618'?
        print(f"The price {price} is within the range 0.5 - 0.786 ({fib_5}, {fib_786})")
    else:
        print(f"The price {price} is outside the range ({min_val}, {max_val})")

get_min_max_data()
print(min_max_vals)
print(min_max_type)

In [ ]:
i = 0
min_val = 9999999999
max_val = 0

while(i != len(min_max_vals)):
    if(min_max_vals[i] in min_max_type[0]):
        if(min_max_vals[i] < min_val):
            print("Updating value: " + str(min_val) + " with new min " + str(min_max_vals[i]))
            min_val = min_max_vals[i]
        else:
            if(max_val > min_val):
                print("Checking possible retracement bt " + str(min_val) + " and " + str(max_val))
                possible_retracement(min_max_vals[i], min_val, max_val)
    else:
        if(min_max_vals[i] > max_val):
            print("Updating value: " + str(max_val) + " with new max " + str(min_max_vals[i]))
            max_val = min_max_vals[i]
    i = i + 1

In [ ]:
'''
[46461.100000000006, 47210.95, 46878.9, 47640.05, 47581.649999999994, 46968.65, 47323.15, 47657.45, 46944.85, 47362.85, 46848.35, 47379.25, 46037.75, 47174.75, 45971.149999999994]
{0: [46461.100000000006, 46878.9, 46968.65, 46944.85, 46848.35, 46037.75, 45971.149999999994], 1: [47210.95, 47640.05, 47581.649999999994, 47323.15, 47657.45, 47362.85, 47379.25, 47174.75]}
'''
def possible_retracement_v2(price, min_val, max_val):
    fib_5, fib_618, fib_786 = fibonacci_levels_v2(min_val, max_val)
    if fib_618 < price <= fib_786:
        #print(f"The price {price} is within the range of 0.618 - 0.786 ({fib_618}, {fib_786})")
        return True
    elif fib_5 < price <= fib_786:  # It seems like 'fib_5' is undefined here. Perhaps it should be 'fib_618'?
        #print(f"The price {price} is within the range 0.5 - 0.786 ({fib_5}, {fib_786})")
        return True
    else:
        #print(f"The price {price} is outside the range ({min_val}, {max_val})")
        return False
print(filtered_df)

i = 0
j = 1
min_val = float("inf")
max_val = 0
size = len(min_max_vals)
trades = []
new_min = False
new_max = False
while(i < size and j < size):
    print(f"i: {i}, j: {j}")
    # Min case
    if(min_max_vals[i] in min_max_type[0]):
        print("new min, setting")
        min_val = min_max_vals[i] # We have a min
        new_min = True
    else:
        print("Searching min, is a max, moving to next element")
        i += 1 # If is max value, move to next element until we get a min

    # Max case
    if(min_max_vals[j] in min_max_type[1]):
        print(f"Checking if {min_max_vals[i]} is greater than {max_val}")
        if(min_max_vals[j] > max_val):
            print("new max, setting")
            max_val = min_max_vals[j] # We have a max
            new_max = True
        else:
            print("Not new max, moving and ignoring it")
            j += 1 # If not new max, move to next element until we get a max
    else:
        print("Searching max, is a min, moving to next element")
        j += 1 # If is min value, move to next element until we get a max

    # Now we have max value, time to search for retracements (min > min_val and min < max_val) from [j+1,len(arr)]
    if(new_min and new_max and min_val < max_val):
        print(f"New Min found {min_val} and New Max found {max_val}. Checking all possible retracements")
        for retracement in range(j+1, size):
            if min_max_vals[retracement] in min_max_type[0]:
                if min_val < min_max_vals[retracement] < max_val:
                    #print(f"Min {min_max_vals[retracement]} is retracement of {min_val} and {max_val}")
                    trades.append([min_val, min_max_vals[retracement], max_val])
                    if(possible_retracement_v2(min_max_vals[retracement], min_val, max_val)):
                        print(f"Fibo found from {min_val} to {max_val} with retracement in {min_max_vals[retracement]}")
            elif min_max_vals[retracement] in min_max_type[1] and min_max_vals[retracement] > max_val: # When new max found, break
                break
        new_max = False
        new_min = False
        i += 1
        j += 1